# Text classification starter

先读取文件，合并数据和target，分成train和test数据集。用算法训练，这里用到sklearn的naive bayes算法（MultinomialNB）来对train数据集进行分类，再用test数据集测试，得到正确率为92.3%。

需要改进的地方：
1. 加入semantics语义，尤其是对中文的，并用nltk
2. 分析为什么有92.3%
3. 研究其他算法的效果
4. 调参

-- alison


## Import libraries

In [1]:
# coding: utf-8 
#encoding=utf-8

import numpy as np
from dateutil import parser
import pandas as pd
from datetime import datetime,date,timedelta
import datetime, warnings, scipy
from dateutil import parser
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
import sklearn.datasets
from sklearn.model_selection import train_test_split

import json 
import csv 
import sys
import os

# import nltk
# nltk.download()


## Import data

In [2]:
def read_data(rootdir, target_name, target_index):
    
    for root, subdirs, files in os.walk(rootdir):
        for file in files:
            path = root+'/'+file
            with open(path, encoding="utf8") as f:
                input_file = f.read()
                data.append(input_file)
                target_names.append(target_name)
                target.append(target_index)

data = []
target_names = []
target = []
read_data("data/classification_data_v3/中性", "中性", 0)
read_data("data/classification_data_v3/买入", "买入", 1)
read_data("data/classification_data_v3/卖出", "卖出", 2)


In [3]:
# combine as dataset
dataset = {
    "data" : data,
    "target_names" :target_names,
    "target" :target
}

# print keys
print(dataset.keys(),"\n")
# print all targets
# print(dataset["target_names"])
# prints target set
set(dataset["target"]) 

dict_keys(['data', 'target_names', 'target']) 



{0, 1, 2}

In [4]:
len(data)

23769

## Train / test data split

In [5]:
data_x = dataset["data"]
data_y = dataset["target"]

train_x, test_x, train_y, test_y = \
    train_test_split(data_x, data_y, test_size = 0.2, random_state = 0)

## Data exploration

### healper functions

In [6]:
# audit dataframe
def table_info(data):
    tab_info=pd.DataFrame(data.dtypes).T.rename(index={0:'column type'})
    tab_info=tab_info.append(pd.DataFrame(data.isnull().sum()).T.rename(index={0:'null values (nb)'}))
    tab_info=tab_info.append(pd.DataFrame(data.isnull().sum()/data.shape[0]*100).T.rename(index={0:'null values (%)'}))
    return tab_info

In [7]:
#prints the first data file
print("\n".join(train_x[0].split("\n")[:])) 

[Table_Page] 
公告点评|半导体 
 证券研究报告 
 
 
[Table_Title]  [Table_Invest] 买入 
华天科技（002185.SZ） 公司评级 
当前价格 13.46 元 
封测行业景气度提升，有望迎来业绩拐点 合理价值 14.65 元 
前次评级 买入 
[Table_Summary] 报告日期 2020-03-01 
核心观点： 
 
 2019 年整体业绩符合预期，短期业绩影响主要由于 2019H1 行业景气 相[Ta对ble市_P场icQ表uot现e]  
度下滑以及一次性财务费用造成。公司披露 2019 年业绩快报，2019
年实现营业收入 81.05 亿元，同比增长 13.80%；归母净利润 2.98 亿 258%
元，同比下滑 23.66%。营收端增长主要是受到收购的 Unisem 并表的 203%
影响；而从盈利端来看，受 2019 年上半年行业深度调整，以及公司要 147%
约收购 Unisem 所新增的银行贷款导致财务费用同比大幅上升等因素 92%
影响，公司 2019 年度归母净利润较上年同期有所下滑。 37%
 封测行业整体景气度回升，公司业绩 2019H2 好转明显。2018 年受半 -18%02/19 04/19 06/19 08/19 10/19 12/19
导体整体周期下行影响，封测行业增速放缓。2019 年二季度起，随着 华天科技 沪深300  
半导体景气度回升，封测行业也明显回暖。华天科技 2019H1 收入为  
38.39 亿元，归母净利润为 0.86 亿元；按照业绩快报披露口径测算， [分Ta析ble师_A：uth or]许 兴军 
2019H2 收入为 42.65 亿元，同比增长 27.86%，环比 2019H1 增长
SAC 执证号：S0260514050002 
 
11.10%。由于 Unisem 在 2019 年 1 月 31 日开始并表，对上下半年影
021-60750532 
响的差异较小，因此即使考虑  Unisem 并表的因素，2019H2 公司收入
 xuxingjun@gf.com.cn 
环比 2019H1 也是增长的。另外，CIS 封测业务处于高景气度下的供
分析师： 王亮 
应紧张，价格上涨叠加产能扩充将有望为公司带来较大的业绩弹性。 
SAC 执证号

## Data preprocessing - pipelined

## Algo1 - Naive base

### Train

In [8]:
# pipeline
text_clf_nb = Pipeline([('vect', CountVectorizer()), # attribute stop_words ("then, the, etc.")
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB(alpha=0.01)), # FitPrior=False, a uniform prior will be used
                    ])

In [9]:
text_clf_nb = text_clf_nb.fit(train_x, train_y)

### Test

In [10]:
predicted = text_clf_nb.predict(test_x)
np.mean(predicted == test_y)

0.8169962137147665

In [13]:
predicted_single = text_clf_nb.predict(test_x[0:1])

In [14]:
print(predicted_single)
print(test_y[0:1])

[1]
[1]


In [11]:
text_clf_nb.score(test_x, test_y)

0.8169962137147665

## Algo2 - Support Vector Machines

### Train

In [13]:
# pipeline
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf-svm', SGDClassifier(loss='hinge', 
                                               penalty='l2',
                                               alpha=1e-3, 
                                               random_state=42)),
                    ])

In [14]:
text_clf_svm = text_clf_svm.fit(train_x, train_y)

### Test

In [15]:
predicted_svm = text_clf_svm.predict(test_x)
np.mean(predicted_svm == test_y)

0.7541018090029449

In [16]:
text_clf_svm.score(test_x, test_y)

0.7541018090029449

## Grid Search

In [17]:
parameters_nb = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
             }

In [18]:
gs_clf_nb = GridSearchCV(text_clf_nb, parameters_nb, n_jobs=-1)
gs_clf_nb = gs_clf_nb.fit(train_x, train_y)

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
gs_clf_nb.best_params_
gs_clf_nb.best_score_

In [ ]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf-svm__alpha': (1e-2, 1e-3),
             }

In [ ]:
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(train_x, train_y)

In [ ]:
gs_clf_svm.best_params_
gs_clf_svm.best_score_

In [ ]:
    public int longestSubarray(int[] nums, int limit) {
        TreeMap map = new TreeMap<>();
        int left = 0;
        int right = 0;
        int res = 0;
        while(right < nums.length) {
            map.put(nums[right], map.getOrDefault(nums[right],0)+1);
            
            while(map.lastKey() - map.firstKey() > limit) {
                map.replace(nums[left], map.get(nums[left])-1);
                if(map.get(nums[left]) == 0) {
                    map.remove(nums[left]);
                }
                left++;
            }
            res = Math.max(res, right - left + 1);
            right++;
        }
        return res;
    }
}